In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Bidirectional

from tensorflow.python.keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from numpy.random import seed


SEED = 123  # used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

df = pd.read_csv("reanalysis.csv")

2024-05-21 10:12:08.194209: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 10:12:08.318078: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 10:12:08.864770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 10:12:10.542769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# SELECT THE TARGET

TARGET = "tas"

df=df.rename(columns={TARGET:'y'})

#Delete all the columns related to a daily scale
for col in df.columns:
    if "_" not in col and col != "y" and col != "time":
        df.drop(col, axis=1, inplace=True)

df["month"] = pd.to_datetime(df["time"]).dt.month
df["hour"] = pd.to_datetime(df["time"]).dt.hour

df.head()

,time,tas_daily,tas_min_daily,tas_max_daily,y,sfcWind_daily,pr_daily,sp_daily,month,hour
0,2000-01-01 00:00:00,295.3574,289.21075,301.2505,295.12326,4.347946,0.018645,100092.25,1,0
1,2000-01-01 01:00:00,295.3574,289.21075,301.2505,294.91876,4.347946,0.018645,100092.25,1,1
2,2000-01-01 02:00:00,295.3574,289.21075,301.2505,294.25897,4.347946,0.018645,100092.25,1,2
3,2000-01-01 03:00:00,295.3574,289.21075,301.2505,291.07550,4.347946,0.018645,100092.25,1,3
4,2000-01-01 04:00:00,295.3574,289.21075,301.2505,290.78757,4.347946,0.018645,100092.25,1,4


In [3]:
# For the target variable get the daily previous value and the daily next value
df["date"] = df["time"].apply(lambda x: x.split(" ")[0])
df_2 = df.copy()
df_2 = df_2[["date", TARGET + "_daily"]].groupby("date").mean()
df_2[TARGET+"_prev_daily"] = df_2[TARGET + "_daily"].shift(1)
df_2[TARGET+"_next_daily"] = df_2[TARGET + "_daily"].shift(-1)
df_2.drop(TARGET + "_daily", axis=1, inplace=True)
df_2.dropna(inplace=True)
df = df.merge(df_2, on="date", how="inner").drop("date", axis=1)

In [4]:
# Convert Categorical column to hot dummy columns
hotencoding1 = pd.get_dummies(df['month'])
hotencoding1 = hotencoding1.add_prefix('month')
hotencoding2 = pd.get_dummies(df['hour'])
hotencoding2 = hotencoding2.add_prefix('hour')

df=df.drop(['month', 'hour'], axis=1)

df=pd.concat([df, hotencoding1, hotencoding2], axis=1)

#Filter the rows with time less than 2015-01-01
df_train = df[df['time'] < '2013-12-31 23:59:00']
df_test = df[df['time'] >= '2013-12-31 23:59:00']

df_train = df_train.set_index('time')
df_test = df_test.set_index('time')

In [5]:
input_X_train = df_train.loc[:, df_train.columns != 'y'].values  # converts df to numpy array
input_X_test = df_test.loc[:, df_test.columns != 'y'].values  
input_y_train = df_train['y'].values
input_y_test = df_test['y'].values

n_features = input_X_train.shape[1]  # number of features

In [6]:
def temporalize(X, y, lookback):
    '''
    Inputs
    X         A 2D numpy array ordered by time of shape: (n_observations x n_features)
    y         A 1D numpy array with indexes aligned with X, i.e. y[i] should correspond to X[i]. Shape: n_observations.
    lookback  The window size to look back in the past records. Shape: a scalar.

    Output
    output_X  A 3D numpy array of shape: ((n_observations-lookback-1) x lookback x n_features)
    output_y  A 1D array of shape: (n_observations-lookback-1), aligned with X.
    '''
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        output_y.append(y[i + lookback + 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

def flatten(X):
    '''
    Flatten a 3D array.

    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.

    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty(
        (X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1] - 1), :]
    return flattened_X

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize

    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])

    return X

In [7]:
lookback = 5
X_train, y_train = temporalize(X=input_X_train, 
                      y=input_y_train, 
                      lookback=lookback)
X_test, y_test = temporalize(X=input_X_test, 
                      y=input_y_test, 
                      lookback=lookback)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=DATA_SPLIT_PCT,
    random_state=SEED)

TIMESTEPS = X_train.shape[1]  # equal to the lookback
N_FEATURES = X_train.shape[2]  # the number of features

In [8]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(X_train))
X_train_scaled = scale(X_train, scaler)
X_valid_scaled = scale(X_valid, scaler)
X_test_scaled =  scale(X_test, scaler)

# Modelo

In [9]:
model = Sequential()
model.add(Input(shape=(TIMESTEPS, N_FEATURES), 
                name='input'))
model.add(
    LSTM(units=16,
         activation='tanh',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_1'))
model.add(Dropout(0.5))
model.add(
    LSTM(units=8,
         activation='tanh',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_2'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=1,
                activation='linear', 
                name='output'))


In [10]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])

In [12]:
history = model.fit(x= np.asarray(X_train_scaled).astype('float32'),
                y=y_train,
                batch_size=128,
                epochs=600,
                validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                y_valid),
                verbose=1).history

Epoch 1/600
  6/767 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 241.8868 - root_mean_squared_error: 15.5516 

2024-05-21 10:36:36.720743: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 86317440 exceeds 10% of free system memory.


761/767 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 256.1035 - root_mean_squared_error: 16.0030

2024-05-21 10:36:42.901816: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21579360 exceeds 10% of free system memory.


767/767 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 256.1047 - root_mean_squared_error: 16.0030 - val_loss: 5.9066 - val_root_mean_squared_error: 2.4304
Epoch 2/600
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 252.6858 - root_mean_squared_error: 15.8960 - val_loss: 4.4922 - val_root_mean_squared_error: 2.1195
Epoch 3/600
767/767 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 250.9610 - root_mean_squared_error: 15.8413 - val_loss: 6.6923 - val_root_mean_squared_error: 2.5869
Epoch 4/600
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 248.4774 - root_mean_squared_error: 15.7631 - val_loss: 4.0192 - val_root_mean_squared_error: 2.0048
Epoch 5/600
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 244.2965 - root_mean_squared_error: 15.6297 - val_loss: 7.1834 - val_root_mean_squared_error: 2.6802
Epoch 6/600
767/767 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 241.1081 - root_mean_squared_error: 15.5275 - val_loss: 3.4486 - val_root_mean_squared_error: 1.8570
Epoch 7/600
767/767 ━━━━━━━━━━━━━━━━━━━━

In [13]:
p = model.predict(np.asarray(X_test_scaled).astype('float32'))
res = pd.DataFrame(p)
filename = "lstm_output_" + TARGET + ".csv"
res.to_csv(filename, index=False)

273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
